In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Criando a base de dados:
base = pd.read_csv('petr4_treinamento.csv')
base = base.dropna()

In [ ]:
# Separando a coluna utilizada nas previsões:
base_treinamento = base.iloc[:, 1:2].values

In [ ]:
# Criando o normalizador:
normalizador = MinMaxScaler(feature_range=(0,1))
base_treinamento_normalizada = normalizador.fit_transform(base_treinamento)

In [ ]:
# Criando os previsores utilizados em cada dia. Neste caso, utilizam-se os 90 dias anteriores para a previsão do preço de cada ação
previsores = []
preco_real = []
for i in range(90, 1242):
    previsores.append(base_treinamento_normalizada[i-90:i, 0])
    preco_real.append(base_treinamento_normalizada[i, 0])

In [ ]:
# Os previsores estão no formato de lista, precisamos transformá-lo em um array
previsores, preco_real = np.array(previsores), np.array(preco_real)

# Colocar a base de dados que servirá como entrada no formato do Keras:
previsores = np.reshape(previsores, (previsores.shape[0], previsores.shape[1], 1))

In [ ]:
# Criando a estrutura de uma rede de 4 camadas:
regressor = Sequential()
regressor.add(LSTM(units = 100, return_sequences = True, input_shape = (previsores.shape[1], 1)))
regressor.add(Dropout(0.3))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.3))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.3))

regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.3))

regressor.add(Dense(units = 1, activation = 'linear'))

regressor.compile(optimizer = 'rmsprop', loss = 'mean_squared_error',
                  metrics = ['mean_absolute_error'])
regressor.fit(previsores, preco_real, epochs = 100, batch_size = 32)

In [ ]:
# Criando a base de teste:
base_teste = pd.read_csv('petr4_teste.csv')
preco_real_teste = base_teste.iloc[:, 1:2].values

In [ ]:
# A rede foi treinada para prever utilizando os 90 dias anteriores.
# Para solucionar este problema, já que nossa base de teste possui somente 22 dados, precisamos executar...
# ... um código que consiga utilizar os 90 dias anteriores de cada um dos dados da base de teste.
base_completa = pd.concat((base['Open'], base_teste['Open']), axis = 0)

# Separar a parte útil:  últimos 90 dados da base de treinamento + a base de teste
entradas = base_completa[len(base_completa) - len(base_teste) - 90:].values

# Colocar no mesmo formato
entradas = entradas.reshape(-1, 1)

# Normalizar
entradas = normalizador.transform(entradas)

# Criando os previsores utilizados no teste:
X_teste = []
for i in range(90, 112):
    X_teste.append(entradas[i-90:i, 0])

# Transformando a lista em um array:
X_teste = np.array(X_teste)

# Colocando no formato exigido pela documentação do Keras:
X_teste = np.reshape(X_teste, (X_teste.shape[0], X_teste.shape[1], 1))

# Testando...
previsoes = regressor.predict(X_teste)
previsoes = normalizador.inverse_transform(previsoes)

# Calculando a previsoes.mean()
preco_real_teste.mean()
    

In [ ]:
previsoes = pd.DataFrame(previsoes)
previsoes
base_teste_comp = pd.concat((previsoes, base_teste['Open'], base_teste['Close']), axis = 1)
base_teste_comp.columns = ['Prevision', 'Open', 'Close']
base_teste_comp
dif_real = base_teste_comp.Close - base_teste_comp.Open
dif_prevista = base_teste_comp.Close - base_teste_comp.Prevision
dif_real, dif_prevista
def ClassY(dif):
    ClasseY=[]
    for element in dif:
        if element>0:
            element = 1
            ClasseY.append(element)
        else:
            element = 0
            ClasseY.append(element)
    return ClasseY
x = ClassY(dif_prevista)
x
y = ClassY(dif_real)
y
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
matriz_petr4 = confusion_matrix(x,y)
TN = matriz_petr4[0,0]
FP = matriz_petr4[0,1]
FN = matriz_petr4[1,0]
TP = matriz_petr4[1,1]
TN, FP, FN, TP, matriz_petr4
print("Relatório de classificação PETR4: \n", classification_report(x, y))  
# ACURÁCIA:
acur = (TP+TN)/float(TP+TN+FP+FN)
print(f'acurácia = {acur}')
# RECALL:
rec = (TP)/float(TP+FN)
print(f'recall = {rec}')
# ESPECIFICIDADE:
esp = (TN)/float(TN+FP)
print(f'especificidade = {esp}')
# PRECISÃO:
prec = (TP)/(TP+FP)
print(f'precisão = {acur}')
# F1-Score:
f1_score = (2*prec*rec)/(prec + rec)
print(f'F1 score = {f1_score}')

In [ ]:
matriz_petr4